Quiero recolectar huellas de la minería en el nor-oeste de Bolivia. No existe un mapa referencial reciente que pueda utilizar para guiar esta exploración. La idea es tomar como puntos de partida reportes del trabajo forense que realizan CEDIB, mongabay y otros, y de ahí explorar libremente. El único límite es la paciencia.  

En este cuaderno construyo intrumentos que me permitan explorar por el territorio nacional via imágenes de Planet NICFI. En particular:

- Una mapa donde pueda observar diferencias claras que podrían ser ocasionadas por actividad minera entre dos puntos en el tiempo.
- Una función para capturar videos de cambios anuales en la vista que muestra el mapa.
- Un índice de capturas y una vista con la ubicación de cada captura para comprender el trayecto y la cobertura hasta el momento.
- Quizás vistas alternativas para indagar más.

In [144]:
# Dependencias

import geemap
import requests
import ee
import json
import os
import datetime as dt
import geopandas as gpd
import pandas as pd
from glob import glob
from IPython.display import display, Markdown, Image, Video

In [4]:
# Inicializar Google Earth Engine
ee.Initialize()

 Defino
 
 - un punto desde donde quiero comenzar a explorar ("Mapiri")
 - el recuadro que quiero consultar ("Bolivia") 
 - y un tiempo desde cuando quiero consultar imágenes (junio de 2016, por limitaciones en la fuente).

In [13]:
def nominatim(query):
    """
    Consulta coordenadas referenciales utilizando 
    nominatim de OSM.
    """
    
    url = f"https://nominatim.openstreetmap.org/search.php?q={query}&polygon_geojson=0&format=jsonv2"
    response = requests.get(url)
    data = response.json()
    
    if len(data) > 0:
        
        d = data[0]
        boundingbox = d['boundingbox']
    
        x = float(d['lon'])
        y = float(d['lat'])

        bbox = ee.Geometry.BBox(
            west = float(boundingbox[2]),
            east = float(boundingbox[3]),
            south = float(boundingbox[0]),
            north = float(boundingbox[1])
        )

        return {
            'center': [y, x],
            'bbox': bbox
        }
    
    else:
        
        return None

In [373]:
bbox = nominatim('Bolivia')['bbox']
center = nominatim('Mapiri, Bolivia')['center']
start = dt.datetime(2016,6,1)

El mapa principal es un panel con 3 vistas:

- Una vista que incluye polígonos de áreas protegidas y un mapa base referencial.
- Una vista a imágenes de Planet en color verdadero en junio de 2016.
- Y una vista a imágenes de Planet en color verdadero en junio de 2023.

In [27]:
def get_nicfi(bbox, start, end=None):
    """
    Retorna imágenes de Planet NICFI en Google Earth Engine
    para el periodo `start` y `end` en un recuadro `bbox` 
    que es un objeto `ee.Geometry.BBox` de GEE. 
    Si `end` no es especificado, consulto la última 
    imagen disponible.
    """
    
    def ee_image2datetime(image):
        return dt.datetime.fromisoformat(image.date().format().getInfo())
    
    def get_first_since(collection, since):
        return (
            collection.
            filter(
                ee.Filter.gt('system:time_end', int(since.timestamp() * 1e3))
            ).
            first()
        )

    collection = (
        ee.
        ImageCollection("projects/planet-nicfi/assets/basemaps/americas").
        filterBounds(bbox)
    )

    start_image = get_first_since(collection, start)

    if end:
        end_image = get_first_since(collection, end)
    else:
        end_image = collection.sort('system:time_start', False).first()
    
    return {
        'start': {
            'image': start_image, 
            'date': ee_image2datetime(start_image)
        },
        'end': {
            'image': end_image, 
            'date': ee_image2datetime(end_image)
        }
    }

In [41]:
def get_wdpa(bbox):
    """
    Retorna una imagen de Google Earth Engine para 
    el World Database on Protected Areas (WDPA).
    """
    
    collection = (
        ee.FeatureCollection("WCMC/WDPA/current/polygons").
        filterBounds(bbox)
    )
    
    image = ee.Image().float().paint(collection, 'REP_AREA')
    
    return image

In [38]:
# Parámetros de visualización para imágenes de Planet y WDPA.

vis = {
    'nicfi': {
        'bands': ['R','G','B'],
        'min':64,
        'max':5454,
        'gamma':1.8
    },
    'wdpa' : {
          'palette': ['2ed033', '5aff05', '67b9ff', '5844ff', '0a7618', '2c05ff'],
          'min': 0.0,
          'max': 1550000.0,
          'opacity': 0.2,
        }
}

In [353]:
def linked_map(bbox, center, start, end=None, zoom=13):
    """
    Construye un mapa con las 3 vistas sincronizadas 
    dentro del recuadro `bbox`, desde el punto `center`, 
    con imágenes de Planet en `start` y `end`.  
    """

    nicfi = get_nicfi(bbox, start, end)
    wdpa = get_wdpa(bbox)
    
    ee_objects = [ wdpa, nicfi['start']['image'], nicfi['end']['image'] ]
    vis_params = [ vis['wdpa'], vis['nicfi'], vis['nicfi'] ]
    labels = [
        'WDPA',
        f'NICFI {nicfi["start"]["date"].strftime("%b %Y")}',
        f'NICFI {nicfi["end"]["date"].strftime("%b %Y")}'
    ]

    Map = geemap.linked_maps(
        rows=1,
        cols=3,
        height='550px',
        ee_objects=ee_objects,
        vis_params=vis_params,
        labels=labels,
        basemap="HYBRID",
        center=center,
        zoom=zoom,
        toolbar_ctrl=True
    )
    
    return Map

Paseo por este mapa y cuando encuentre algo interesante tomo un video de timelapse anual entre 2016 y 2023.

In [177]:
def get_view_state(view):
    """
    Retorna coordenadas de la vista en un objeto de mapa. 
    """
    
    state = view.get_manager_state()['state']
    model = [state[k] for k in state.keys() if state[k]['model_name'] == 'LeafletMapModel'][-1]
    state_params =  {i: model['state'][i] for i in ['north', 'south', 'east', 'west', 'center', 'zoom']}
    return state_params

In [204]:
def nicfiTimelapse(view, start, directory='timelapse', fps=3):
    """
    Produce un video de timelapse anual desde `start` 
    para el recuadro en la vista del mapa `view`, que guarda
    en `directory` con un nombre que incluye información
    sobre el tiempo y ubicación de cobertura.
    """

    state = get_view_state(view)
    
    bbox = ee.Geometry.BBox(
        west = state['west'],
        east = state['east'],
        south = state['south'],
        north = state['north']
    )
    
    nicfi = (
        ee.
        ImageCollection("projects/planet-nicfi/assets/basemaps/americas").
        filterBounds(bbox)
    )
    
    domain = {
        'start': start.strftime('%Y-%m-%d'),
        'now': dt.datetime.now().strftime('%Y-%m-%d')
    }
    
    filename = f"nicfi_{domain['start']}_{domain['now']}_{state['center'][0]}_{state['center'][1]}"
    
    timelapse = geemap.create_timelapse(
        collection=nicfi,
        start_date=domain['start'],
        end_date=domain['now'],
        region=bbox,
        frequency='year',
        bands=['R','G','B'],
        frames_per_second=fps,
        mp4=True,
        out_gif=f'{directory}/{filename}.gif',
        font_type='/home/m/.local/share/fonts/Archivo/static/Archivo-Medium.ttf',
        font_size=15,
        font_color="#f4f4f4",
        add_progress_bar=False,
        title="© Planet NICFI",
        title_xy=('2%', '2%'),
        text_xy=('2%', '5%'),
        vis_params=vis['nicfi']
    )
    
    os.remove(f'{directory}/{filename}.gif')
    display(Video(f'{directory}/{filename}.mp4'))

Luego de tomar varios videos, quiero asegurarme de no pasar por el mismo lugar demasiadas veces y guiar mejor mi ruta. Para eso creo un índice con la posición de cada captura y quizás datos adicionales como si se encuentra en un área protegida. Utilizo ese índice para mostrar un mapa junto a la posición actual en el mapa. 

In [150]:
def get_areas_protegidas():
    """
    Produce una tabla simple con los polígonos de todas las áreas protegidas
    según el Viceministerio de Tierras.
    """
    
    gdfs = []
    for f in glob('resources/areas*'):
        nombre = f.split('.')[0].split('_')[-1]
        gdf = gpd.read_file(f)
        area_col = [col for col in gdf.columns if 'nombre' in col][0]
        gdf.insert(0, 'tipo', nombre)
        gdf.rename(columns={area_col: 'nombre'}, inplace=True)
        gdfs.append(gdf[['tipo', 'nombre', 'geometry']])
    return gpd.GeoDataFrame(pd.concat(gdfs), geometry='geometry')

areas_protegidas = get_areas_protegidas()

In [151]:
areas_protegidas

,tipo,nombre,geometry
0,municipales,Huaripampa,"MULTIPOLYGON (((-68.12761 -16.42619, -68.12780..."
1,municipales,Huallatani Pampa,"MULTIPOLYGON (((-67.96064 -16.43700, -67.96042..."
2,municipales,Laguna Concepcion,"MULTIPOLYGON (((-61.23603 -17.39050, -61.23603..."
3,municipales,Laguna Quirusillas,"MULTIPOLYGON (((-63.98854 -18.29465, -63.98782..."
4,municipales,Serrania Sararenda-Cuevo,"MULTIPOLYGON (((-63.55209 -20.24037, -63.55127..."
...,...,...,...
22,departamentales,Bruno Racua,"MULTIPOLYGON (((-65.57370 -9.83905, -65.56971 ..."
23,departamentales,Area de proteccion del Quebracho Colorado,"MULTIPOLYGON (((-62.38027 -20.99960, -62.25982..."
24,departamentales,Area de proteccion del Pino del Cerro,"MULTIPOLYGON (((-64.74788 -21.89838, -64.78004..."
25,departamentales,Altamachi,"MULTIPOLYGON (((-66.86342 -15.93578, -66.67905..."


In [179]:
def create_index(directory='timelapse'):
    """
    Construye un índice de todas las capturas.
    """
    
    entries = []
    
    for video in glob(f'{directory}/*.mp4'):
        
        v = video.replace('.mp4', '').split('_')
        entries.append({
            'filename': video,
            'capture': v[2],
            'y': float(v[3]),
            'x': float(v[4])
        })
        
    df = pd.DataFrame(entries)
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.x,df.y, crs='EPSG:4326'))
    gdf = gdf.drop(columns=['x', 'y'])
    gdf = gpd.tools.sjoin(gdf, areas_protegidas[['nombre', 'geometry']], how='left').drop(columns='index_right')
    gdf.rename(columns={'nombre': 'area_protegida'}, inplace=True)
    
    return gdf

In [246]:
def plot_current(index):
    """
    Produce un mapa simple con la ubicación de cada captura
    y la posición actual en el mapa.
    """
    
    state = get_view_state(view)
    m = index.explore( 
        color='blue',
        style_kwds={'stroke': True, 'color': '#9C9BB5', 'weight': .5}, 
        marker_kwds={'radius': 5}, 
        legend_kwds={'colorbar': False},
    )
    state_coords = gpd.GeoDataFrame(geometry=gpd.points_from_xy([state['center'][1]], [state['center'][0]]))
    state_coords.explore(
        m=m, 
        color='red', 
        marker_kwds={'radius': 5}, 
        legend_kwds={'colorbar': False}
    )
    return m

Y adicionalmente un simple enlace que me lleva a la ubicación del mapa en la vista satelital de Google Maps. Esta vista tiene una resolución mejor que Planet NICFI.

In [332]:
def googleMaps():
    state = get_view_state(view)
    return display(Markdown(
        f'[map](https://www.google.com/maps/@{state["center"][0]},{state["center"][1]},2258m/data=!3m1!1e3?entry=ttu)'
    ))

El panel con 3 mapas sincronizados ...

In [374]:
view = linked_map(bbox, center, start, end=dt.datetime(2023,6,1))
view

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

La ubicación actual en la vista satelital de Google ...

In [356]:
googleMaps()

[map](https://www.google.com/maps/@-11.675016230544056,-67.73272991180421,2258m/data=!3m1!1e3?entry=ttu)

Tomo una captura cuando encuentro algo interesante ...

In [367]:
nicfiTimelapse(view, start)

Generating URL...
Please wait ...
The GIF image has been saved to: /home/m/studio/code/mining/timelapse/nicfi_2016-06-01_2023-11-07_-14.669041072847948_-68.97856235504152.gif


Y un mapa para revisar dónde he tomado capturas hasta ahora ...

In [383]:
index = create_index()
plot_current(index)

El índice con todas las capturas ...

In [384]:
index

,filename,capture,geometry,area_protegida
0,timelapse/nicfi_2016-06-01_2023-11-07_-13.9162...,2023-11-07,POINT (-68.83713 -13.91624),Madidi
1,timelapse/nicfi_2016-06-01_2023-11-07_-15.5750...,2023-11-07,POINT (-68.03196 -15.57509),NaN
2,timelapse/nicfi_2016-06-01_2023-11-07_-14.9907...,2023-11-07,POINT (-68.56623 -14.99072),Apolobamba
3,timelapse/nicfi_2016-06-01_2023-11-07_-15.2282...,2023-11-07,POINT (-68.37386 -15.22821),Apolobamba
4,timelapse/nicfi_2016-06-01_2023-11-07_-15.1985...,2023-11-07,POINT (-68.80596 -15.19856),Apolobamba
...,...,...,...,...
117,timelapse/nicfi_2016-06-01_2023-11-07_-14.5956...,2023-11-07,POINT (-68.56398 -14.59567),Madidi
118,timelapse/nicfi_2016-06-01_2023-11-07_-14.2460...,2023-11-07,POINT (-68.53930 -14.24604),Madidi
119,timelapse/nicfi_2016-06-01_2023-11-07_-14.3461...,2023-11-07,POINT (-68.56025 -14.34618),Madidi
120,timelapse/nicfi_2016-06-01_2023-11-07_-14.0003...,2023-11-07,POINT (-68.74920 -14.00033),Madidi


Que al final guardo para construir otras cosas ... 

In [385]:
with open('index.json', 'w+') as f:
    f.write(
        index[['filename', 'capture', 'area_protegida', 'geometry']].to_json()
    )